In [1]:
!pip install thefuzz

In [2]:
!pip install python-Levenshtein

In [3]:
!pip install kneed

In [4]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
from thefuzz import process, fuzz
from collections import defaultdict
from kneed import KneeLocator
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.impute import KNNImputer
from sklearn.metrics.pairwise import cosine_similarity
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import warnings
import joblib
warnings.filterwarnings('ignore')

1. Data Preprocessing

In [6]:
#load dataset
data = pd.read_csv('C:/Users/Owinyez/OneDrive/Documentos/Projos/Research Analysis/FAOSTAT_data_en_6-30-2025.csv')

#display sample
data.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,404,Kenya,5510,Production,2960.01,Beeswax,2021,2021,t,2596.80,I,Imputed value,NaN
1,QCL,Crops and livestock products,404,Kenya,5510,Production,2960.01,Beeswax,2022,2022,t,2605.43,I,Imputed value,NaN
2,QCL,Crops and livestock products,404,Kenya,5510,Production,2960.01,Beeswax,2023,2023,t,2614.06,I,Imputed value,NaN
3,QCL,Crops and livestock products,404,Kenya,5510,Production,22241.01,Butter of cow milk,2021,2021,t,19158.81,I,Imputed value,NaN
4,QCL,Crops and livestock products,404,Kenya,5510,Production,22241.01,Butter of cow milk,2022,2022,t,17460.28,I,Imputed value,NaN


1.1 Exploring the dataset

In [9]:
#number of rows and columns
data.shape

(216, 15)

In [10]:
#data types and null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       216 non-null    object 
 1   Domain            216 non-null    object 
 2   Area Code (M49)   216 non-null    int64  
 3   Area              216 non-null    object 
 4   Element Code      216 non-null    int64  
 5   Element           216 non-null    object 
 6   Item Code (CPC)   216 non-null    float64
 7   Item              216 non-null    object 
 8   Year Code         216 non-null    int64  
 9   Year              216 non-null    int64  
 10  Unit              216 non-null    object 
 11  Value             216 non-null    float64
 12  Flag              216 non-null    object 
 13  Flag Description  216 non-null    object 
 14  Note              0 non-null      float64
dtypes: float64(3), int64(4), object(8)
memory usage: 25.4+ KB


In [11]:
#check unique values in Elemet Code
data['Element Code'].nunique()

9

9 Element Codes

In [12]:
#check the elemets
data['Element'].value_counts()

Production                       114
Producing Animals/Slaughtered     60
Yield/Carcass Weight              24
Milk Animals                      12
Yield                              3
Laying                             3
Name: Element, dtype: int64

In [13]:
#check unique value in Item Code (CPC)
data['Item Code (CPC)'].nunique()

41

41 unique item codes

In [14]:
#check the items
data['Item'].value_counts()

Hen eggs in shell, fresh                                                15
Meat of goat, fresh or chilled                                           9
Meat of sheep, fresh or chilled                                          9
Meat of rabbits and hares, fresh or chilled                              9
Meat of pig with the bone, fresh or chilled                              9
Meat of chickens, fresh or chilled                                       9
Meat of cattle with the bone, fresh or chilled                           9
Meat of camels, fresh or chilled                                         9
Raw hides and skins of cattle                                            6
Raw hides and skins of sheep or lambs                                    6
Raw milk of camel                                                        6
Goat fat, unrendered                                                     6
Raw milk of cattle                                                       6
Raw milk of goats        

In [15]:
#check flags
data['Flag'].value_counts()

I    106
A     81
E     29
Name: Flag, dtype: int64

In [16]:
#check flag description
data['Flag Description'].value_counts()

Imputed value      106
Official figure     81
Estimated value     29
Name: Flag Description, dtype: int64

2. Data Cleaning

2.1 Handling Missing Values

In [18]:
data.isnull().sum()

Domain Code           0
Domain                0
Area Code (M49)       0
Area                  0
Element Code          0
Element               0
Item Code (CPC)       0
Item                  0
Year Code             0
Year                  0
Unit                  0
Value                 0
Flag                  0
Flag Description      0
Note                216
dtype: int64

The Column "Note" is empty, drop columns
Other columns to drop:'Domain Code', 'Domain', 'Area Code (M49)', 'Area', 'Year Code'


2.1 Dealing with Duplicates

In [27]:
#check duplicate rows based on all columns
print(f"Number of duplicate rows: {data.duplicated()}")

Number of duplicate rows: 0      False
1      False
2      False
3      False
4      False
       ...  
211    False
212    False
213    False
214    False
215    False
Length: 216, dtype: bool


No duplicates

In [29]:
#save the clean dataset to new CSV
data.to_csv('C:/Users/Owinyez/OneDrive/Documentos/Projos/Research Analysis/FOSTAS Data Cleaned.csv')